# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[@pedrohdaher ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'bradesco'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 401

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

print('Esperamos trabalhar no diretório')
print(os.getcwd())
dados= pd.read_excel('bradesco.xlsx',sep=',')
dados.head()

Esperamos trabalhar no diretório
C:\Users\Pedro Daher\Downloads


,Treinamento,Relevante
0,rt @bradesco: a história de orgulho dos gaúcho...,N
1,i'm at agência bradesco in são paulo w/ @livka...,N
2,@renatin_pereira bradesco?,N
3,@bradesco é \nre_marques94@hotmail.com 😊,N
4,uns meses atrás eu reparei que o quando eu vou...,S


In [33]:
#Lista de palavras e sinais que não vamos utilizar:

limpa = ['@','1','2','3','4','5','6','7','8','9','0',' de ', ' do ',' dos ', ' para ', ' pra ',' e ',' que ', ',', '"', '(', ')','&','[',']','.',',', '#','*','%','-','_','=','/','\\','\n']

#Extraindo o arquivo que iremos utilizar

empresa = pd.read_excel('bradesco.xlsx',sep=',')

#For para fazer a limpeza, irá remover todas as stop words

for carac in limpa: 
    empresa['Treinamento'] = empresa['Treinamento'].str.replace(carac, ' ')  

    
#Leitura da Coluna de Classificação (frequencia de relativos e nao relativos)

Freq = (empresa['Relevante'].value_counts(sort=False, normalize=True)*100).round(decimals=1)  

#São as probabilidades de serem relevantes e de serem não relevantes (pr e pn)

ProbR = Freq['S'] 
ProbN = Freq['N'] 


#Frequencia absoluta de cada palavra
Abs = empresa.Treinamento.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0) 

#Frequencia absoluta de cada palavra relevante
AbsR = empresa.Treinamento[company.Relevante == 'S'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0) 

#Frequencia absoluta de cada palavra não relevante
AbsN = empresa.Treinamento[company.Relevante == 'N'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0) 

#Relevantes e não relevantes (Total)

nr = tr[0]  
nn = tn[0]

#Palavras sem repetições

n = len(t)

nr = nr.item()
nn = nn.item()

nr = int(nr)
nn = int(nn)

freqr = pd.DataFrame(tr)
freqn = pd.DataFrame(tn)

#Lista de frequencia de cada palavra em cada categoria
lf = []  

ln = [] 


for i in range(len(freqr)):
    x = (((freqr.iat[i,0])+1)/(nr+n))*pr
    lf.append(x)
    
for i in range(len(freqn)):
    x = (((freqn.iat[i,0])+1)/(nn+n))*pn
    ln.append(x)
    
#Lista de palavras
lr = freqr.index.tolist() 

#Lista de palavras
l = freqn.index.tolist() 
dr = {}
dn = {}

for i in range(len(freqr)):
    dr[lr[i]] = lf[i]
    
for i in range(len(freqn)):
    dn[l[i]] = ln[i]

teste2 = pd.read_excel('bradesco.xlsx',sep=',', sheetname="Teste")

#Variáveis necessarias(pr, pn, dn, dr)

teste = pd.read_excel('bradesco.xlsx',sep=',', sheetname="Teste")
ui = 0
for index, row in teste.iterrows():
    y1 = 1
    y2 = 1
#Tweet
    p = row['Teste'] 

#Lista com as palavras do tweet    
    l = p.split() 
    for i in l:
        if i in dr:
            y1 *= (dr[i])
        if i in dn:
            y2 *= (dn[i])
    if y1>y2:
        teste.set_value(index, 'Relevante', 'S')
        ui = ui+1        
        
    else:
        teste.set_value(index, 'Relevante', 'N')

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('resultado.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
teste.to_excel(writer, sheet_name='Testado')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
teste.head()

,Teste,Relevante
0,odeio o bradesco com todas as minhas forças.,N
1,trabalhadores assinam acordos específicos com ...,S
2,13 minutos esperando um atendimento do bradesc...,N
3,ex-bancários do bec assinam acordo com o brade...,N
4,rt @umvesgo: e até quem me vê na fila do brade...,N


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [34]:
certo1 = 0
certo2 = 0 
errado1 = 0
errado2 = 0



for w in range(0,len(teste)):
    if teste2['Relevante'][w] == 'S' and teste['Relevante'][w] == 'N':
        certo1 += 1
    if teste2['Relevante'][w] == 'S' and teste['Relevante'][w] == 'S':
        certo2 += 1
    if teste2['Relevante'][w] == 'N' and teste['Relevante'][w] == 'S':
        errado1 += 1
    if teste2['Relevante'][w] == 'N' and teste['Relevante'][w] == 'N':
        errado2 += 1
        
        
#Porcentagens

PC1 = (certo1/200)*100
PC2 = (certo2/200)*100
PC3 = (errado1/200)*100
PC4 = (errado2/200)*100

print(PC1,PC2,PC3,PC4)
print("Porcentagem de positivos falsos (marcados como relevante mas não são relevantes):{0}".format(PC1))
print("Porcentagem de positivos verdadeiros (marcados como relevante e são relevantes):{0}".format(PC2))
print("Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes):{0}".format(PC3))
print("Porcentagem de negativos falsos (marcados como irrelevante mas são relevantes):{0}".format(PC4))
print(len(teste2))

23.5 1.0 15.5 60.0
Porcentagem de positivos falsos (marcados como relevante mas não são relevantes):23.5
Porcentagem de positivos verdadeiros (marcados como relevante e são relevantes):1.0
Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes):15.5
Porcentagem de negativos falsos (marcados como irrelevante mas são relevantes):60.0
200


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


## Conclusão

As mídias sociais hoje funcionam como um grande espaço de feedback dos clientes para empresas. Diversas delas já utilizam Twitter e Facebook como plataformas de SAC (Serviço de Atendimento ao Cliente), sendo assim, eles podem atender os clientes de modo rápido e dinâmico. Além dos mesmos responderem qualquer Tweet ou Post que se refira a algum problema encontrado com qualquer serviço da empresa, as redes sociais podem funcionar como um grande termômetro da reputação da empresa diante do público.

Sendo assim, ao construir o classificador de mensagens relevantes ou não é um primeiro passo nessa grande analise de toda informação criada e construída sobre a empresa. Posts falando mal de serviços, ou parabenizando algum serviço da empresa reunidos em um software que lê tudo e classifica para você usando como base o algoritimo já conhecido de Naive-Bayes foi o objetivo desse projeto. 

O algoritimo quebra as palvras e baseado em um banco de dados anterior, lhe diz a probabilidade de cada palavra ser relevante ou não, a combinação de palavras (o tweet em si) terá uma probabilidade X de pertencer à categoria de relevantes e uma probabilidade Y de pertencer à categoria de irrelevantes, devido ao produto das probabilidades de cada palvra do tweet.

Baseado nisso, nosso programa corta certas palavras e, principalmente, caracteres que consideramos irrelevantes para avaliação das probabilidades antes ditas. Sendo assim, a empresa que avaliamos foi o Bradesco, e os produtos mais citados foram agência bancária e uma pequena parte o Internet Banking, o que já nos seleciona parte do público que comenta frequentemente sobre o banco. Problemas em agência foram fortemente citados pelos clientes do Twitter mas ao mesmo tempo as mensagens agressivas são complicadas de se avaliar devido aos diferentes modos de usar palavras para compor um tweet negativo em relação aos tweets positivos. Nesse sentido, nosso programa teve imenso problema ao avaliar a relevância de tweets para alertar o banco, apenas 1% bateu com a nossa prévia feita à mão. Ao mesmo tempo, em relação a discartar tweets que não tem relevância nosso programa obteve certa efetividade ao acertar em 60% com a nossa base anterior.

Mensagens com piadas, ironia e sarcasmo são definitivamente o problema para um software. Estes trabalham com variáveis, com equações, é muito complicado ensinar interpretação de texto para um computador, e ainda mais considerando o vasto território nacional e as variações linguisticas, além é claro, de erros ortográficos dos mais diversos que necessitam avaliação também.

O objetivo final é ter um programa com 100% de acerto em relação à nossa interpretação e a interpretação do programa, para isso é necessario um machine learning que não é rápido e nem à curto prazo, devido a variação da lingua em um curto intervalo de tempo. Para expandir, precisamos analisar mais dados em menos tempo e cada vez mais obter um programa que tenha um acerto maior, emojis são lançados quase todo mês e necessitam de adaptaçào ao nosso software.